In [ ]:
#
# Google Drive に pota フォルダーと pota/maps フォルダーを作成
# hunter_pars.csv：POTAサイトからハンター情報をダウンロードし、potaフォルダーにコピー
#
# ライブラリのインポート
import folium
import pandas as pd

# Googleドライブのマウント
from google.colab import drive
drive.mount("/content/drive")

# ハント情報の読み込み 対象を日本に絞る
hunter=pd.read_csv("/content/drive/My Drive/pota/hunter_parks.csv")
hunter_jp = hunter.query("Reference.str.contains('JA-', regex=True)", engine="python").reset_index()
hunter_jp.head()

Mounted at /content/drive


,index,DX Entity,Location,HASC,Reference,Park Name,First QSO Date,QSOs
0,2,Japan,Aichi,JP-AI,JA-0129,Mikawa Wan Quasi-National Park,2022-12-01,1
1,3,Japan,Aichi,JP-AI,JA-1465,Owari Koiki Ryokudo Prefectural Park,2023-03-24,1
2,4,Japan,Aichi,JP-AI,JA-1466,Aichi Health Village Prefectural Park,2023-03-24,1
3,5,Japan,Aichi,JP-AI,JA-1926,Aichi Education and Sports Foundation Prefectu...,2023-03-12,1
4,6,Japan,Akita,JP-AK,JA-1060,Akita Shirakami Prefectural Nature Park,2023-04-09,1


In [ ]:
# 公園情報の読み込み
parks = pd.read_json("https://tyjcode.github.io/y001/parks-latest.json")
parks.head()

,Reference,name,日本語名,locationDesc,locationName,都道府県,latitude,longitude
0,JA-0001,Rishiri-Rebun-Sarobetsu,利尻礼文サロベツ国立公園,JP-HK,Hokkaido,北海道,45.1942,141.239
1,JA-0002,Shiretoko,知床国立公園,JP-HK,Hokkaido,北海道,44.1650,145.205
2,JA-0003,Akan-Mashu,阿寒摩周国立公園,JP-HK,Hokkaido,北海道,43.4362,144.103
3,JA-0004,Kushiroshitsugen,釧路湿原国立公園,JP-HK,Hokkaido,北海道,43.1383,144.445
4,JA-0005,Daisetsuzan,大雪山国立公園,JP-HK,Hokkaido,北海道,43.6717,142.907


In [ ]:
# ハント情報に公園情報をマージ
hunter_map = pd.merge(hunter_jp, parks, on="Reference")
hunter_map.head()

,index,DX Entity,Location,HASC,Reference,Park Name,First QSO Date,QSOs,name,日本語名,locationDesc,locationName,都道府県,latitude,longitude
0,2,Japan,Aichi,JP-AI,JA-0129,Mikawa Wan Quasi-National Park,2022-12-01,1,Mikawa Wan,三河湾国定公園,JP-AI,Aichi,愛知県,34.8155,137.233
1,3,Japan,Aichi,JP-AI,JA-1465,Owari Koiki Ryokudo Prefectural Park,2023-03-24,1,Owari Koiki Ryokudo,尾張広域緑道,JP-AI,Aichi,愛知県,35.3341,136.931
2,4,Japan,Aichi,JP-AI,JA-1466,Aichi Health Village Prefectural Park,2023-03-24,1,Aichi Health Village,あいち健康の森公園,JP-AI,Aichi,愛知県,34.9987,136.948
3,5,Japan,Aichi,JP-AI,JA-1926,Aichi Education and Sports Foundation Prefectu...,2023-03-12,1,Aichi Education and Sports Foundation,愛知県一宮総合運動場(いちい信金スポーツセンター),JP-AI,Aichi,愛知県,35.3020,136.846
4,6,Japan,Akita,JP-AK,JA-1060,Akita Shirakami Prefectural Nature Park,2023-04-09,1,Akita Shirakami,秋田白神県立自然公園,JP-AK,Akita,秋田県,40.4200,140.120


In [ ]:
# 都道府県でグルーピング 件数をカウント
hunter_pref_group = hunter_jp.groupby("Location").count().iloc[:,:1].reset_index()
hunter_pref_group.head()

,Location,index
0,Aichi,4
1,Akita,3
2,Chiba,14
3,Ehime,1
4,Fukui,1


In [ ]:
# ハントしていない公園を抽出 (非アクティブを含む)
non_hunter = pd.merge(parks, hunter_jp, on="Reference", how="outer", indicator=True)
non_hunter_map = non_hunter[non_hunter["_merge"] == "left_only"].iloc[:,:8]
non_hunter_map.head()

,Reference,name,日本語名,locationDesc,locationName,都道府県,latitude,longitude
0,JA-0001,Rishiri-Rebun-Sarobetsu,利尻礼文サロベツ国立公園,JP-HK,Hokkaido,北海道,45.1942,141.239
1,JA-0002,Shiretoko,知床国立公園,JP-HK,Hokkaido,北海道,44.1650,145.205
2,JA-0003,Akan-Mashu,阿寒摩周国立公園,JP-HK,Hokkaido,北海道,43.4362,144.103
3,JA-0004,Kushiroshitsugen,釧路湿原国立公園,JP-HK,Hokkaido,北海道,43.1383,144.445
4,JA-0005,Daisetsuzan,大雪山国立公園,JP-HK,Hokkaido,北海道,43.6717,142.907


In [ ]:
# 全国地図にプロット
# max_widthを入れないと縦書きになる
map = folium.Map(location=[35.7503, 139.523], zoom_start=6)

# ハントしてない公園
for i, r in non_hunter_map.iterrows():
  name_pop = folium.Popup(f'{r["Reference"]}<br/>{r["日本語名"]}', max_width=1000)
  folium.Marker(location=[r["latitude"], r["longitude"]], popup=name_pop,icon=folium.Icon(color='red')).add_to(map)

# ハントした公園
for i, r in hunter_map.iterrows():
  name_pop = folium.Popup(f'{r["Reference"]}<br/>{r["日本語名"]}<br/>{r["QSOs"]}', max_width=1000)
  folium.Marker(location=[r["latitude"], r["longitude"]], popup=name_pop).add_to(map)

# 地図の作成
map.save("/content/drive/My Drive/pota/maps/map_parks.html")

In [ ]:
# ハントした都道府県別に地図を作成
#

for j, h in hunter_pref_group.iterrows():
  print(h["Location"],h["index"])

  # 都道府県毎に抽出
  non_hunter_map_pref = non_hunter_map.query('locationName=="{}"'.format(h["Location"])).reset_index()
  hunter_map_pref = hunter_map.query('Location=="{}"'.format(h["Location"])).reset_index()

  # 最初の公園を中心に地図を作成
  map2 = folium.Map(location=[hunter_map_pref.loc[0,"latitude"], hunter_map_pref.loc[0,"longitude"]], zoom_start=10)

  # ハントしてない公園
  for i, r in non_hunter_map_pref.iterrows():
    name_pop = folium.Popup(f'{r["Reference"]}<br/>{r["日本語名"]}', max_width=1000)
    folium.Marker(location=[r["latitude"], r["longitude"]], popup=name_pop,icon=folium.Icon(color="red")).add_to(map2)

  # ハントした公園   
  for i, r in hunter_map_pref.iterrows():
    name_pop = folium.Popup(f'{r["Reference"]}<br/>{r["日本語名"]}<br/>{r["QSOs"]}', max_width=1000)
    folium.Marker(location=[r["latitude"], r["longitude"]], popup=name_pop).add_to(map2)

  # 地図の作成
  map2.save(f'/content/drive/My Drive/pota/maps/map_parks2_{h["Location"]}_{h["index"]}.html')


Aichi 4
Akita 3
Chiba 14
Ehime 1
Fukui 1
Fukuoka 2
Fukushima 2
Gifu 2
Gumma 11
Hiroshima 4
Hyogo 2
Ibaraki 10
Ishikawa 2
Kagoshima 1
Kanagawa 22
Kyoto 2
Mie 6
Miyagi 6
Miyazaki 4
Nagano 6
Nara 4
Niigata 1
Oita 1
Okayama 1
Okinawa 3
Osaka 6
Saitama 29
Shiga 2
Shimane 1
Shizuoka 9
Tochigi 8
Tokushima 2
Tokyo 81
Toyama 3
Wakayama 1
Yamagata 13
Yamaguchi 2
Yamanashi 9
